
<h1> Datto RMM - End of Life OS and Software Report </h1>

# DataFrame Creation from API
## Prepare for DataFrame Creation
### Set Export Folder

In [ ]:
# Data Shaping
import pandas as pd
import datetime as dt

# File Handling and Export
import json

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Import Modules, Create env Variables

In [ ]:
from ts_api_connections import datto_rmm_api
env_file = f'{git_folder}/config/env.ini'

datto = datto_rmm_api.DattoRMM(env_file=env_file)
df_devices = datto.create_devices_dataframe()

In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read(env_file)

# import and assign secrets from env.ini
config_dict = {"base_uri": config["dattormm"]["base_uri"],
               "api_key": config["dattormm"]["api_key"],
               "api_secret": config["dattormm"]["api_secret"]
               }

In [ ]:
"https://concord.centrastage.net/csm/profile/downloadAgent/[REDACTED_UUID]"

In [ ]:
df_devices[df_devices['uid'] == "39e63353-1512-80b3-8b0c-2bc5d5bfccdf"]

In [ ]:
df_devices.to_html("D:/exports/export.html")

# Initial Metrics and CSV Creation

## All Fields

In [ ]:
df_devices.to_csv(export_folder + 'all_fields_' + str(current_time) + '[REDACTED]/.csv',index=False)

## Reboot Issues

### Empty 'Last Reboot' Field

In [ ]:
df_devices[df_devices['lastReboot'].fillna('Missing') == 'Missing'].to_csv(export_folder + 'last_reboot_isnull_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Reboot > 60 Days

In [ ]:
df_devices[df_devices['noReboot30Days'] == 1].to_csv(export_folder + 'no_reboot_30_days_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Reboot Since Last Year

In [ ]:
df_devices[df_devices['lastReboot'].dt.year == dt.datetime.utcnow().year - 1].to_csv(export_folder + 'no_reboot_current_yr_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Reboot in 2 Years

In [ ]:
df_devices[df_devices['lastReboot'].dt.year == dt.datetime.utcnow().year - 2].to_csv(export_folder + 'no_reboot_2yrs_' + str(current_time) + '[REDACTED]/.csv',index=False)

## DattoRMM Audit Issues

### Empty 'Last Audit' Field

In [ ]:
df_devices[df_devices['lastAuditDate'].fillna('Missing') == 'Missing'].to_csv(export_folder + 'last_audit_isnull_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Audit > 7 days

In [ ]:
df_devices[df_devices['noAudit7Days'] == 1].to_csv(export_folder + 'no_audit_7days_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Audit > 30 days

In [ ]:
df_devices[df_devices['noAudit30Days'] == 1].to_csv(export_folder + 'no_audit_30days_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Audit Since Last Year

In [ ]:
df_devices[df_devices['lastAuditDate'].dt.year == dt.datetime.utcnow().year - 1].to_csv(export_folder + 'no_audit_current_yr_' + str(current_time) + '[REDACTED]/.csv',index=False)

## Patch Status

### Install Error Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusInstallError'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_InstallError_isonline_' + str(current_time) + '[REDACTED]/.csv',index=False)

### Reboot Required Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusRebootRequired'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_RebootRequired_isonline_' + str(current_time) + '[REDACTED]/.csv',index=False)

### No Data Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusNoData'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_NoData_isonline_' + str(current_time) + '[REDACTED]/.csv',index=False)

## Patch Percentage

### Patch Status Percentage under 95%

In [ ]:
df_devices[df_devices['patchStatusPercent'] < 95].to_csv(export_folder + 'patchStatus_Under_95_' + str(current_time) + '[REDACTED]/.csv',index=False)

# Service Delivery Reports
(ticket creation report)

## MS Patching Service Report Shaping Criteria

In [ ]:
df = pd.read_csv(f'{git_folder}[REDACTED]/.csv')
ms_patching = {'siteNames': list(df['siteName'].dropna()), 'column': list(df['column'].dropna())}

## Create Filtered Service Delivery DataFrame

In [ ]:
df_service_delivery = df_devices[df_devices['siteName'].isin(ms_patching['siteNames'])]
df_service_delivery = df_service_delivery[ms_patching['column']]

### Service Delivery - Offline > 30 Days

In [ ]:
df_service_delivery[df_service_delivery['offline30Days'] == 1].to_csv(export_folder + 'service_delivery_offline_30days_' + str(current_time) + '[REDACTED]/.csv')

### Service Delivery - Patch Status Percentage under 95% and Online > 30 days

In [ ]:
df_service_delivery[(df_service_delivery['patchStatusPercent'] < 95) & (df_service_delivery['offline30Days'] == 0)].to_csv(export_folder + 'service_delivery_patchStatus_Under_95_isOnline_' + str(current_time) + '[REDACTED]/.csv')

### Service Delivery - Device EOL (out of compliance)

In [ ]:
df_service_delivery[df_service_delivery['isEOL'] == 1].to_csv(export_folder + 'service_delivery_eol_devices_' + str(current_time) + '[REDACTED]/.csv')